# HealthNotebook 



Welcome to HealthNotebook!

This is a notebook for health data analysis.

In [ ]:
# Import modules
import modules.data_module as data_module
import numpy as np
import pickle
import warnings
# Supress warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

First, we would ask you to fill this short survey. Run the cell below to see the survey. Once you are done, click 'Submit' to save your answers.

In [ ]:
import modules.questionnaire_module

In [ ]:
# Get user data from the survey
user_data = modules.questionnaire_module.get_data()
# Load data
df = data_module.get_data()

Next we will compare your answers to your peers. In the plots below, you can see how your answers compare to our data.

In [ ]:
# Plot sleep time
data_module.plot(dataset = df,
                 userdata= user_data,
                 plotted_variables = ['Sleeptime (h)'])  

In [ ]:
# Plot sleep latency as a function of stress
data_module.plot(dataset = df,
                 userdata= user_data,
                 plotted_variables = ['Stress', 'Latency']) 

In [ ]:
# Plot sleep latency as a function of steps
data_module.plot(dataset = df,
                 userdata= user_data,
                 plotted_variables = ['Steps', 'Latency'])

In [ ]:
# Plot sleep time as a function of stress
data_module.plot(dataset = df,
                 userdata= user_data,
                 plotted_variables = ['Stress', 'Sleeptime (h)'])

In [ ]:
# Plot restfulness as a function of food healthiness
data_module.plot(dataset = df,
                 userdata= user_data,
                 plotted_variables = ['Food Healthiness', 'Restfulness'])

Below we will predict your sleep time and sleep latency based on your answers to the survey. By sleep latency we mean the time it takes to fall asleep after going to bed. We will also tell you how much you can improve your sleep by increasing your daily step count and reducing your level of stress.

In [ ]:
with open('./data/models/covid_model_sleeptime.pkl', 'rb') as f:
    model_sleeptime = pickle.load(f)

with open('./data/models/covid_model_latency.pkl', 'rb') as f:
    model_latency = pickle.load(f)

with open("./data/models/uhs_model_rest.pkl", "rb") as f:
    model_rest = pickle.load(f)

# Predict sleep time and latecy for the user
x = np.array([user_data["Steps"], user_data["Stress"]])
pred_sleep = model_sleeptime.predict(x.reshape(-1, 2))
pred_latency = model_latency.predict(x.reshape(-1, 2))
print("Predicted sleeptime", round(pred_sleep[0], 2), "(h) and predicted latency", round(pred_latency[0], 2), "(min).")

# Make another prediction with decreased stress and added exercise
# (higher stress level actually means lower stress)
x = np.array([user_data["Steps"] + 2000, user_data["Stress"] + 7])
pred_sleep_alt = model_sleeptime.predict(x.reshape(-1, 2))
pred_latency_alt = model_latency.predict(x.reshape(-1, 2))
diff_sleep = (pred_sleep_alt[0] - pred_sleep[0]) * 60
diff_latency = pred_latency_alt[0] - pred_latency[0]
print("By moderately decresing your stress level and adding steps to your day you can achieve the following changes.")
print("Change in your sleep time:", round(diff_sleep, 1), "minutes.")
print("Change in your sleep latency:", round(diff_latency, 1), "minutes.")

Finally, we will predict how much you can improve the restfulness of your sleep by improving your diet.

In [ ]:
pred_rest = model_rest.predict(np.array([user_data["Food Healthiness"]]).reshape(-1, 1))

if user_data["Food Healthiness"] == 2:
    text = "Congratulations! You have healthy food habits."
elif user_data["Food Healthiness"] == 1:
    pred_rest_alt = model_rest.predict(np.array([user_data["Food Healthiness"] + 1]).reshape(-1, 1))
    change = 100 * (pred_rest_alt - pred_rest) / pred_rest
    change = round(change[0][0], 2)
    text = f"You can improve the restfulness of your sleep {change} % by improving your diet."
else:
    pred_rest_alt = model_rest.predict(np.array([user_data["Food Healthiness"] + 2]).reshape(-1, 1))
    change = 100 * (pred_rest_alt - pred_rest) / pred_rest
    change = round(change[0][0], 2)
    text = f"You can improve the restfulness of your sleep {change} % by improving your diet."

print(text)